In [1]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist=input_data.read_data_sets("MNIST")

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST\train-labels-idx1-ubyte.gz
Extracting MNIST\t10k-images-idx3-ubyte.gz
Extracting MNIST\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
X_train=mnist.train.images
y_train=mnist.train.labels
X_test=mnist.test.images
y_test=mnist.test.labels

In [3]:
from IPython.display import clear_output, Image, display, HTML
import numpy as np
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [32]:
import tensorflow as tf
#MNIST
n_inputs=28*28 
n_hidden1=100
n_hidden2=50
n_outputs=10

n_epochs=60
batch_size=100

X=tf.placeholder(tf.float32,shape=(None,n_inputs),name="X")
y=tf.placeholder(tf.int64,shape=(None),name="y")

#or use the handy tf dense function
tf.reset_default_graph()
X=tf.placeholder(tf.float32,shape=(None,n_inputs),name="X")
y=tf.placeholder(tf.int64,shape=(None),name="y")

#apply l1 regularizer and he init method
from functools import partial
he_init = tf.contrib.layers.variance_scaling_initializer(mode="FAN_AVG") # he init method
scale=0.001
my_dense=partial(tf.layers.dense,activation=tf.nn.elu,
                 kernel_regularizer=tf.contrib.layers.l1_regularizer(scale),
                 kernel_initializer=he_init)

with tf.name_scope("dnn"):
    dropout_rate=0.5
    training=tf.placeholder_with_default(False,shape=(),name="training")
    hidden1=my_dense(X,n_hidden1,name="hidden1",kernel_initializer=he_init,activation=tf.nn.elu) #use ELU
    #dropout1
    hidden1_drop=tf.layers.dropout(hidden1,dropout_rate,training=training)
    hidden2=my_dense(hidden1_drop,n_hidden2,name="hidden2",kernel_initializer=he_init,activation=tf.nn.elu) #ELU
    #dropout
    hidden2_drop=tf.layers.dropout(hidden2,dropout_rate,training=training)
    logits=my_dense(hidden2_drop,n_outputs, kernel_regularizer=tf.contrib.layers.l1_regularizer(scale),
                 kernel_initializer=he_init,name="outputs")
    
with tf.name_scope("loss"):
    xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss=tf.reduce_mean(xentropy,name="loss")
    
learning_rate=0.01
with tf.name_scope("train"):
    #learning_rate scheduling
#     initial_learning_rate=0.1
#     decay_steps=10000
#     decay_rate=1/2
#     global_step=tf.Variable(0,trainable=False,name="global_step")
#     learning_rate=tf.train.exponential_decay(initial_learning_rate,global_step,
#                                              decay_steps,decay_rate)
    #use NAG
    #Big boost
#     optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,
#                                          momentum=0.9,use_nesterov=True)
    
    #use Adam
    optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op=optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct=tf.nn.in_top_k(logits,y,1)
    accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))
init=tf.global_variables_initializer()
saver=tf.train.Saver()


with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            X_batch,y_batch=mnist.train.next_batch(batch_size)
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch,training:True}) # dropout=True
        acc_train=accuracy.eval(feed_dict={X:X_batch,y:y_batch})
        acc_test=accuracy.eval(feed_dict={X:mnist.test.images,y:mnist.test.labels})
        print(epoch,"Train accuracy: ",acc_train," Test accuracy: ",acc_test,end="\r")
    save_path=saver.save(sess,"./mnist_final.ckpt")